In [85]:
import sqlite3
from sqlite3 import Error
import pandas as pd

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.row_factory = sqlite3.Row  #this for getting the column names
        #print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def df_from_query(query):

    cur = conn.cursor()
    
    try:
        cur.execute(query)
    except Error as e:
        if isinstance(e, sqlite3.OperationalError):
            print('OperationalError: {}'.format(e.args)) #print the error statement
            return None  #if query has error
    rows = cur.fetchall()
    columns = [_[0] for _ in cur.description]
    if len(rows) == 0:  #if result has no rows
        df = pd.DataFrame(columns = columns)
    if len(rows)>0:     #if result is multiple row
        df = pd.DataFrame(rows, columns=columns)
    return df





In [93]:
database = "sqlite_db_pythonsqlite.db"

#T stands for tasks in SQL Tasks Tier 1

T10 = '''SELECT f.name, 
       SUM(CASE WHEN m.memid != 0 THEN f.membercost*b.slots              ELSE f.guestcost*b.slots END) as Revenue
  FROM Bookings as b
  JOIN Facilities as f
       ON b.facid = f.facid
  JOIN Members as m
       ON b.memid = m.memid
 GROUP BY f.name
 HAVING Revenue < 1000
 ORDER BY Revenue DESC'''


T11 = '''SELECT m.surname as MemSurname, m.firstname as MemFirstname, 
                r.surname as RecSurname, r.firstname as RecFirstname
           FROM Members as m
           JOIN Members as r
             ON m.recommendedby = r.memid 
          ORDER BY MemSurname, MemFirstname, 
                RecSurname,RecFirstname'''

T12 = '''SELECT f.name,  n.surname || ', ' || n.firstname as Name, SUM(b.slots)/2 as FacilUseHrs
  FROM Bookings as b
  JOIN Facilities as f
       ON b.facid = f.facid
  JOIN Members as m
       ON b.memid = m.memid
  JOIN Members as n
       ON m.memid = n.memid  /*making sure different members with same names are not lumped together*/
  WHERE m.memid > 0 
  GROUP BY f.name, m.memid, n.surname || ', ' || n.firstname
  ORDER BY f.name ASC, FacilUseHrs DESC'''

T13 = '''SELECT f.name,  strftime('%m', b.starttime) || '/' || strftime('%Y', b.starttime) as MonthYear, SUM(b.slots)/2 as FacilUseHrs
  FROM Bookings as b
  JOIN Facilities as f
       ON b.facid = f.facid
  JOIN Members as m
       ON b.memid = m.memid
  WHERE m.memid > 0 
  GROUP BY f.name, strftime('%m', b.starttime) || '/' || strftime('%Y', b.starttime)
  ORDER BY f.name ASC, FacilUseHrs DESC'''

# create a database connection
conn = create_connection(database)
                         
task_dict = {'T10':[T10], 
             'T11':[T11],
             'T12':[T12],
             'T13':[T13]}

for k,v in task_dict.items():
    with conn: 
        query = v[0]
        print(query, '\n')
        v.append(df_from_query(query))
        if v[1] is not None:
            print(v[1].head())
            print('Task {} executed succesfully.\n'.format(t))
        else: 
            print('Error in {}'.format(k))
               

SELECT f.name, 
       SUM(CASE WHEN m.memid != 0 THEN f.membercost*b.slots              ELSE f.guestcost*b.slots END) as Revenue
  FROM Bookings as b
  JOIN Facilities as f
       ON b.facid = f.facid
  JOIN Members as m
       ON b.memid = m.memid
 GROUP BY f.name
 HAVING Revenue < 1000
 ORDER BY Revenue DESC 

            name  Revenue
0     Pool Table      270
1  Snooker Table      240
2   Table Tennis      180
Task T10 executed succesfully.

SELECT m.surname as MemSurname, m.firstname as MemFirstname, 
                r.surname as RecSurname, r.firstname as RecFirstname
           FROM Members as m
           JOIN Members as r
             ON m.recommendedby = r.memid 
          ORDER BY MemSurname, MemFirstname, 
                RecSurname,RecFirstname 

  MemSurname MemFirstname RecSurname RecFirstname
0      Bader     Florence   Stibbons       Ponder
1      Baker         Anne   Stibbons       Ponder
2      Baker      Timothy    Farrell       Jemima
3     Boothe          Tim    

In [72]:
a = {'34':[234], 'df':[33]}
print(a)
df_from_query('''select * from members ''')

{'34': [234], 'df': [33]}


,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05
5,5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09 10:44:09
6,6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,,2012-07-15 08:52:55
7,7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4,2012-07-25 08:59:12
8,8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3,2012-07-25 16:02:35
9,9,Stibbons,Ponder,"5 Dragons Way, Winchester",87630,(833) 160-3900,6,2012-07-25 17:09:05
